In [17]:
# Import the libraries to load the model
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import sequence

In [18]:
#Load the imdb dataset word index
word_index = imdb.get_word_index()
reversed_word_index = dict([(value, key) for (key, value) in word_index.items()])   


In [19]:
model = load_model('imdb_model_simplernn.h5')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (32, 500, 128)         │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_4 (SimpleRNN)        │ (32, 500, 64)          │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (32, 500, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_5 (SimpleRNN)        │ (32, 64)               │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (32, 64)               │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (32, 1)                │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,304,835 (4.98 MB)

 Trainable params: 1,304,833 (4.98 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [20]:
model.get_weights()

[array([[ 0.02789769, -0.04482168, -0.01933881, ..., -0.02106926,
         -0.03121301,  0.01444407],
        [ 0.02108775,  0.00456652,  0.03380434, ...,  0.00870889,
          0.00116659,  0.00409119],
        [-0.02809718, -0.01023606,  0.03970403, ..., -0.02723616,
          0.06720366, -0.02911876],
        ...,
        [-0.00334939, -0.09264677,  0.06236063, ...,  0.09409677,
         -0.01683828, -0.0814027 ],
        [-0.01196061, -0.02286817,  0.06171647, ...,  0.02311245,
         -0.03978271, -0.06004012],
        [-0.07312105, -0.05919327,  0.05859747, ...,  0.02372818,
         -0.05447814,  0.00302362]], dtype=float32),
 array([[ 0.16212568,  0.13568945, -0.03354223, ..., -0.01994672,
         -0.00497482, -0.01630603],
        [-0.01651543, -0.14108743, -0.15021351, ..., -0.02731824,
          0.08920421, -0.2046706 ],
        [-0.02702621, -0.02616062,  0.17381112, ...,  0.10729037,
         -0.04829021,  0.1497876 ],
        ...,
        [-0.04517854,  0.04301297, -0.0

In [21]:
# Helper funstions
# function to decode reviews
def decode_review(encoded_review):
    return ' '.join([reversed_word_index.get(i-3, '?') for i in encoded_review])  

In [22]:
# Function to preprocess user input
def preprocess_text(text):
    '''
    Converts the text to lowercase using text.lower() to make the process case-insensitive.
    Splits the lowercase text into individual words using the .split() method, creating a list of words.
    Purpose: Ensures consistent text processing and tokenization by treating "The" and "the" as the same word.
    '''
    words = text.lower().split()
    
    '''
    Iterates over the list of words.
    For each word, retrieves its corresponding integer index from a dictionary named word_index using word_index.get(word, 2).
    If the word is not found in the word_index dictionary, it assigns a default value of 2 (a placeholder for unknown words).
    Adds 3 to each index value to account for special tokens like <PAD>, <START>, or <UNK> (customarily reserved in text preprocessing).
    Purpose: Transforms the text into a sequence of integers that represent the words, making it suitable for input to machine learning models.
    
    '''
    encoded_review = [word_index.get(word, 2) + 3 for word in words]
    
    '''
    Passes the encoded_review to sequence.pad_sequences() to ensure all sequences have the same length.
    maxlen=500: Specifies that the output sequence should be exactly 500 words long.
    padding='post': Adds zeros (<PAD> tokens) to the end of the sequence if it’s shorter than 500 words.
    truncating='post': Removes words from the end of the sequence if it’s longer than 500 words.
    Wraps encoded_review in a list ([encoded_review]) because pad_sequences expects a list of sequences.
    Purpose: Ensures a fixed sequence length for all reviews, which is required for deep learning models.
    
    '''
    padded_review = sequence.pad_sequences([encoded_review], maxlen=500, padding='post', truncating='post')
    return padded_review

In [23]:
# Prediction function
def predict_sentiment(review):
    
    '''
    Calls the preprocess_text function to preprocess the user input review.
    Passes the preprocessed review to the model for prediction.
    Returns the predicted sentiment (0 for negative, 1 for positive).'''
    
    preprocessed_input= preprocess_text(review)
    '''
    Calls the model.predict() method to make a prediction on the preprocessed review.
    Returns the predicted sentiment (0 for negative, 1 for positive).'''
    
    prediction = model.predict(preprocessed_input)
    
    sentiment = 'Positive' if prediction[0][0] > 0.5 else 'Negative'
    
    return sentiment, prediction[0][0]

In [25]:
# Example review for prediction
example_review ="This movie was fantastic! The acting was great and the plot was thrilling."

sentiment,score=predict_sentiment(example_review)

print(f'Review: {example_review}')
print(f'Sentiment: {sentiment}')
print(f'Prediction Score: {score}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Review: This movie was fantastic! The acting was great and the plot was thrilling.
Sentiment: Positive
Prediction Score: 0.9105536937713623
